In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.tokenize.treebank import TreebankWordDetokenizer
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
df = pd.read_csv("Master/04_Analysis/04_02_ensi_skills_extraction/04_01_custom_analysis/04_01_cleaned_and_trunslated_dataset.csv", encoding = "utf-8")
df.head(5)

,jobTitle,jobTitle_eng,Company,Location,Branche,CompanyGroup,Rating,jobDescription,jobDescription_eng,org_lang,est_minSalary,est_maxSalary,postDate,Unnamed: 13
0,abschlussarbeit controlling business intelligence,final thesis controlling business intelligence,lidl stiftung & co kg,neckarsulm,groß- & einzelhandel,lidl,NaN,linde material handling rhein ruhr entwickelt ...,linde material handling rhein ruhr develops hi...,de,NaN,NaN,22-05-2022,0
1,abteilungsleiter datenmanagement data warehous...,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,NaN,location is flexible job purpose the role is f...,location is flexible job purpose the role is f...,en,103890.0,140900.0,25-05-2022,1
2,accumulation data analyst in accumulation and ...,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,NaN,data engineer join us let care for tomorrow at...,data engineer join us let care for tomorrow at...,en,NaN,NaN,11-04-2022,2
3,accumulation data engineer in accumulation and...,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,NaN,join us let care for tomorrow at allianz globa...,join us let care for tomorrow at allianz globa...,en,NaN,NaN,11-04-2022,3
4,actuarial data scientist,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,wir übernehmen verantwortung für das nachhalti...,we take responsibility for the sustainable man...,de,NaN,NaN,25-05-2022,4


In [3]:

# 1. Drop jobDescription, jobTitle
# 2. Rename jobDescription_eng, jobTitle_eng to jobDescription, jobTitle
# 3. Rename Branche to Industry
# 4. Rename Unnamed 14 to text_id
# 5. Fill NaN rows with -1,0,N.A.
# 6. Drop duplicates with jobDescription and CompanyGroup
# 7. Fill NaN rows with -1,0,N.A.
# 8. Create new column salary_low, salary_high and salary_mean
# 9. Drop est_minSalary and est_maxSalary



def preprocess_data(dataframe):

    dataframe = dataframe.drop(columns=['jobDescription','jobTitle'])
    dataframe = dataframe.rename(columns={'jobDescription_eng': 'jobDescription','jobTitle_eng': 'jobTitle','Branche': 'Industry','Unnamed: 13': 'text_id'})
    dataframe.fillna({'est_minSalary':-1,'est_maxSalary':-1, 'Rating':0, 'Industry': 'N.A.'},inplace=True)
    dataframe = dataframe.drop_duplicates(subset=['jobDescription','CompanyGroup'])
    dataframe.fillna({'est_minSalary':-1,'est_maxSalary':-1, 'Rating':0, 'Industry': 'N.A.'},inplace=True)
    dataframe['salary_low'] = dataframe['est_minSalary'].astype(int)
    dataframe['salary_high'] = dataframe['est_maxSalary'].astype(int)
    dataframe['salary_mean'] = (dataframe["salary_high"] + dataframe['salary_low'])/2
    dataframe = dataframe.drop(columns=['est_minSalary', 'est_maxSalary'], axis=1)
    return dataframe

    
df = preprocess_data(df)
df.describe(include="all")

,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean
count,2420,2420,2419,2420,2420,2420.000000,2420,2420,2420,2420.000000,2420.000000,2420.000000,2420.000000
unique,1670,1493,350,94,1376,NaN,2406,2,67,NaN,NaN,NaN,NaN
top,data scientist,deutsche bahn ag,berlin,it-services & it-consulting,deutsche bahn ag,NaN,corporate description as a communication speci...,de,11-04-2022,NaN,NaN,NaN,NaN
freq,119,48,330,269,48,NaN,2,1408,425,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,0.817769,NaN,NaN,NaN,1209.500000,19057.414876,22942.407438,20999.911157
std,NaN,NaN,NaN,NaN,NaN,1.641058,NaN,NaN,NaN,698.738148,30491.833844,37807.489389,33699.184961
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,-1.000000,-1.000000,-1.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,604.750000,-1.000000,-1.000000,-1.000000
50%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1209.500000,-1.000000,-1.000000,-1.000000
75%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1814.250000,53856.000000,65144.750000,57881.500000


In [4]:
# The first line of code drops any duplicate rows in the 'jobDescription' column of the df dataframe. The second line of code displays summary statistics for all columns in the dataframe, including the 'jobDescription' column.

df.drop_duplicates(subset=['jobDescription'],inplace=True)
df.describe(include="all")

,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean
count,2406,2406,2405,2406,2406,2406.000000,2406,2406,2406,2406.000000,2406.000000,2406.000000,2406.000000
unique,1661,1488,348,93,1370,NaN,2406,2,67,NaN,NaN,NaN,NaN
top,data scientist,deutsche bahn ag,berlin,it-services & it-consulting,deutsche bahn ag,NaN,linde material handling rhein ruhr develops hi...,de,11-04-2022,NaN,NaN,NaN,NaN
freq,117,48,329,267,48,NaN,1,1399,419,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,0.811887,NaN,NaN,NaN,1206.591438,19023.150457,22890.132585,20956.641521
std,NaN,NaN,NaN,NaN,NaN,1.636003,NaN,NaN,NaN,699.027987,30468.146089,37767.432535,33664.325355
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,-1.000000,-1.000000,-1.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,601.250000,-1.000000,-1.000000,-1.000000
50%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1204.500000,-1.000000,-1.000000,-1.000000
75%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1811.750000,53638.000000,65119.000000,57875.750000


In [5]:

# The first line of code defines a list of stop words, which are words that are commonly used in the English language but don't hold a lot of meaning, such as "the" or "and". 
# The second line of code defines a list of characters that should be removed from the job titles, such as punctuation marks and digits. The third line of code defines a function that takes in a dataframe as an argument. 
# This function tokenizes the job titles, which means it splits them up into individual words. It also removes any stop words or punctuation marks. Finally, it detokenizes the job titles, which means it puts them back together into a single string. 
# The fourth line of code applies this function to the dataframe. The fifth line of code prints the number of rows in the dataframe. The sixth line of code prints the first two rows of the dataframe.stop_words = stopwords.words('english')


stop_words = stopwords.words('english')
to_remove = stop_words + list(string.punctuation) + list(string.digits)
def tokenazing_job_titles(dataframe):
    dataframe['job_title_token'] = dataframe['jobTitle'].apply(word_tokenize).apply(lambda x: [item for item in x if item not in to_remove])
    dataframe['job_title_clean'] = dataframe['job_title_token'].apply(TreebankWordDetokenizer().detokenize)
    dataframe['job_title_clean'] = dataframe['job_title_clean'].str.replace('/', ' ')
    dataframe['job_title_clean'] = dataframe['job_title_clean'].str.replace('-', ' ')
    dataframe['job_title_clean'] = dataframe['job_title_clean'].str.replace('.', ' ')
    return dataframe
df = tokenazing_job_titles(df)
print(df.shape[0])
df.head(2)

2406


,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean
0,final thesis controlling business intelligence,lidl stiftung & co kg,neckarsulm,groß- & einzelhandel,lidl,0.0,linde material handling rhein ruhr develops hi...,de,22-05-2022,0,-1,-1,-1.0,"[final, thesis, controlling, business, intelli...",final thesis controlling business intelligence
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890,140900,122395.0,"[department, head, data, management, data, war...",department head data management data warehouse...


In [9]:
# This code is used to determine the seniority level of a given job title. It does this by extracting certain keywords from the job title that indicate the seniority level, and thenreplace those keywords with labels indicating the seniority level. 
# For example, if the job title contains the word "vice president", it will be replaced with the label "exec".

# The code first defines a function called "seniority_level" that takes in a dataframe as an input. It then creates a new column in the dataframe called "level", which contains the seniority level labels.

# Next, the code uses the "str.extract" function to extract keywords from the "job_title_clean" column that indicate the seniority level. The extracted keywords are then replaced with labels indicating the seniority level.

# Finally, the code fills in any missing values in the "level" column with the label "mid" and removes any rows that contain the word "student" or "internship" in the "jobTitle" column.



def seniority_level(dataframe):
    
    dataframe['level'] = dataframe['job_title_clean'].str.extract('(vice president|vp|senior|junior|principal|president|associate|sr|sr.|jr|jr.|director|chief|manager|lead|head|entry|trainee|trainees|student|internship)')
    dataframe['level'] = dataframe['level'].replace("(vice president|vp)", 'exec', regex = True)
    dataframe['level'] = dataframe['level'].replace("(chief|director)", 'head', regex = True)
    dataframe['level'] = dataframe['level'].replace("(lead|principal|manager)", 'manager', regex = True)
    dataframe['level'] = dataframe['level'].replace("(senior|sr|sr.)", 'senior', regex = True)
    dataframe['level'] = dataframe['level'].replace("(associate|staff)", 'mid', regex = True)
    dataframe['level'] = dataframe['level'].replace("(junior|jr|entry|trainee|trainees)", 'junior', regex = True)
    dataframe['level'] = dataframe['level'].replace("(student|internship)", 'student', regex = True)
    dataframe.level.fillna('mid', inplace = True)
    dataframe = dataframe.query('level != "student"')
    dataframe = dataframe[dataframe['jobTitle'].str.contains("thesis")==False]

    return dataframe
df = seniority_level(df)
print(F"Number of jobs with seniority level:\n {df.level.value_counts()}")
print(F"Number of jobs with seniority level:\n {df.shape[0]}")
df.head(2)

Number of jobs with seniority level:
 mid        1601
senior      437
junior      170
manager     114
head         20
Name: level, dtype: int64
Number of jobs with seniority level:
 2342


,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean,level
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890,140900,122395.0,"[department, head, data, management, data, war...",department head data management data warehouse...,head
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,-1,-1,-1.0,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid


In [10]:
# The code below is creating a new column in the dataframe called 'role', and then using the .str.extract() function to pull out the roles from the 'job_title_clean' column. 
# The roles that are being extracted are:
#                                       'data scientist', 'data engineer', 'data analysis', 'business analyst', 'bi', 'intelligence', 'data analyst', 'web', 'machine learning engineer', 
#                                       'data science', 'data engineering', 'analyst', 'analytics', 'research', 'scientist', 'science', 'engineer', 'data warehouse architect', 'data warehouse specialist', 
#                                       'data architect'.



df['role'] = df['job_title_clean'].str.extract('(data scientist|data engineer|data analysis|business analyst|bi|intelligence|data analyst|web|machine learning engineer|\
                                                data science|data engineering|analyst|analytics|research|scientist|science|engineer|data warehouse architect|data warehouse specialist|data architect)')


In [12]:
# This code is extracting the role column from a dataframe, and replacing the various job titles with 3 simplified job titles - 'data engineer', 'data analyst', and 'data scientist'. 
# It then prints the number of rows in the dataframe, and returns the first 5 rows.



df.dropna(subset=['role'], inplace=True)
def extracting_role(dataframe):
    dataframe['role'] = dataframe['role'].replace("(data engineer|engineer|machine learning engineer|data warehouse architect|data warehouse specialist|data architect)", 'data engineer', regex = True)
    dataframe['role'] = dataframe['role'].replace("(data analyst|analytics|analyst|web|data analysis|bi|intelligence)", 'data analyst', regex = True)
    dataframe['role'] = dataframe['role'].replace("(data scientist|data science|research|scientist|science)", 'data scientist', regex = True)
    return dataframe

df = extracting_role(df)
df_original_salary = df.copy()
print(df.shape[0])
df.head(5)


2311


,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean,level,role
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890,140900,122395.0,"[department, head, data, management, data, war...",department head data management data warehouse...,head,data engineer
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,-1,-1,-1.0,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid,data analyst
3,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,join us let care for tomorrow at allianz globa...,en,11-04-2022,3,-1,-1,-1.0,"[accumulation, data, engineer, accumulation, c...",accumulation data engineer accumulation catast...,mid,data engineer
4,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,we take responsibility for the sustainable man...,de,25-05-2022,4,-1,-1,-1.0,"[actuarial, data, scientist]",actuarial data scientist,mid,data scientist
5,actuarial data scientist actuar,concordia versicherungsgesellschaft ag,hannover,versicherungen,concordia versicherungsgesellschaft ag,0.0,hirschau central full time about us conrad con...,de,11-04-2022,5,-1,-1,-1.0,"[actuarial, data, scientist, actuar]",actuarial data scientist actuar,mid,data scientist


In [13]:
# 1. This code creates a dataframe called df_base which only includes the columns: text_id, jobTitle, role, level, jobDescription, Location, Company, Industry
# 2. Creates a dataframe called df_original_salary which only includes the columns: text_id, salary_low, salary_high, salary_mean
# 3. Creates a copy of the df dataframe called df_furher_analysis



df_base = df[['text_id','jobTitle','role','level','jobDescription','Location','Company','Industry']]
df_base.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/00_tableau_base_data.xlsx',index=False)
df_original_salary = df_original_salary[['text_id','salary_low','salary_high','salary_mean']]
df_original_salary.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/02_tableau_original_salary.xlsx',index=False)


df_furher_analysis = df.copy()
df_furher_analysis.head()

,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean,level,role
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890,140900,122395.0,"[department, head, data, management, data, war...",department head data management data warehouse...,head,data engineer
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,-1,-1,-1.0,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid,data analyst
3,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,join us let care for tomorrow at allianz globa...,en,11-04-2022,3,-1,-1,-1.0,"[accumulation, data, engineer, accumulation, c...",accumulation data engineer accumulation catast...,mid,data engineer
4,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,we take responsibility for the sustainable man...,de,25-05-2022,4,-1,-1,-1.0,"[actuarial, data, scientist]",actuarial data scientist,mid,data scientist
5,actuarial data scientist actuar,concordia versicherungsgesellschaft ag,hannover,versicherungen,concordia versicherungsgesellschaft ag,0.0,hirschau central full time about us conrad con...,de,11-04-2022,5,-1,-1,-1.0,"[actuarial, data, scientist, actuar]",actuarial data scientist actuar,mid,data scientist


In [14]:
# replacing any Salary with -1 with the average salary of respective role, level and location


# This code replaces the -1 values in salary_mean with the mean salary for the same role, level, and location.
# The code first creates lists of unique jobs, ranks, and locations where salary_mean is -1.
# It then loops through each job, rank, and location in the lists.
# For each job, rank, and location, the code replaces the -1 values in salary_mean, salary_low, and salary_high with the mean salary for that job, rank, and location.



def replacing_salary_role_level_location(dataframe):
    jobs = dataframe[dataframe.salary_mean == -1].role.unique().tolist()
    rank = dataframe[dataframe.salary_mean == -1].level.unique().tolist()
    cities = dataframe[dataframe.salary_mean == -1].Location.unique().tolist()
    for i in jobs:
        for j in rank:
            for c in cities:
                dataframe['salary_mean'].mask(((dataframe['salary_mean'] == -1) & (dataframe['role'] == i) & (dataframe['level'] == j) & (dataframe['Location'] == c)), dataframe[(dataframe['role'] == i) & (dataframe['salary_mean'] != -1 ) & (dataframe['level'] == j)& (dataframe['Location'] == c)].salary_mean.mean(), inplace=True)
                dataframe['salary_low'].mask(((dataframe['salary_low'] == -1) & (dataframe['role'] == i) & (dataframe['level'] == j) & (dataframe['Location'] == c)), dataframe[(dataframe['role'] == i) & (dataframe['salary_low'] != -1 ) & (dataframe['level'] == j)& (dataframe['Location'] == c)].salary_low.mean(), inplace=True)
                dataframe['salary_high'].mask(((dataframe['salary_high'] == -1) & (dataframe['role'] == i) & (dataframe['level'] == j) & (dataframe['Location'] == c)), dataframe[(dataframe['role'] == i) & (dataframe['salary_high'] != -1 ) & (dataframe['level'] == j)& (dataframe['Location'] == c)].salary_high.mean(), inplace=True)
    return dataframe
df_furher_analysis = replacing_salary_role_level_location(df_furher_analysis)
df_furher_analysis.head(5)

,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean,level,role
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890.0,140900.000000,122395.000000,"[department, head, data, management, data, war...",department head data management data warehouse...,head,data engineer
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,64538.0,82615.833333,73576.916667,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid,data analyst
3,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,join us let care for tomorrow at allianz globa...,en,11-04-2022,3,73200.0,73931.500000,73565.750000,"[accumulation, data, engineer, accumulation, c...",accumulation data engineer accumulation catast...,mid,data engineer
4,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,we take responsibility for the sustainable man...,de,25-05-2022,4,NaN,NaN,NaN,"[actuarial, data, scientist]",actuarial data scientist,mid,data scientist
5,actuarial data scientist actuar,concordia versicherungsgesellschaft ag,hannover,versicherungen,concordia versicherungsgesellschaft ag,0.0,hirschau central full time about us conrad con...,de,11-04-2022,5,64172.0,84575.000000,74373.500000,"[actuarial, data, scientist, actuar]",actuarial data scientist actuar,mid,data scientist


In [15]:
# this function is used to replace the salary with the average salary of the role and level 


#This code is replacing the NaN values in the salary columns with the mean values of the salaries of the same roles and level.

#Firstly, it replaces the NaN values with -1, so that it can easily distinguish which values have been changed to the mean.

#It then creates two lists, jobs_2 and rank_2, which contain all of the roles and levels that have salary values of -1.

#For each role in jobs_2 and each level in rank_2, it replaces the salary values of -1 with the mean salary value of that role and level.



def replacing_salary_role_level(dataframe):
    dataframe['salary_mean'] = dataframe['salary_mean'].replace(np.nan, -1)
    dataframe['salary_low'] = dataframe['salary_low'].replace(np.nan, -1)
    dataframe['salary_high'] = dataframe['salary_high'].replace(np.nan, -1)
    jobs_2 = dataframe[dataframe.salary_mean == -1].role.unique().tolist()
    rank_2 = dataframe[dataframe.salary_mean == -1].level.unique().tolist()
    for i in jobs_2:
        for j in rank_2: 
            dataframe['salary_mean'].mask(((dataframe['salary_mean'] == -1) & (dataframe['role'] == i) & (dataframe['level'] == j)), dataframe[(dataframe['role'] == i) & (dataframe['salary_mean'] != -1 ) & (dataframe['level'] == j)].salary_mean.mean(), inplace=True)
            dataframe['salary_low'].mask(((dataframe['salary_low'] == -1) & (dataframe['role'] == i) & (dataframe['level'] == j)), dataframe[(dataframe['role'] == i) & (dataframe['salary_low'] != -1 ) & (dataframe['level'] == j)].salary_low.mean(), inplace=True)
            dataframe['salary_high'].mask(((dataframe['salary_high'] == -1) & (dataframe['role'] == i) & (dataframe['level'] == j)), dataframe[(dataframe['role'] == i) & (dataframe['salary_high'] != -1 ) & (dataframe['level'] == j)].salary_high.mean(), inplace=True)
    return dataframe
df_furher_analysis = replacing_salary_role_level(df_furher_analysis)
df_furher_analysis.head(5)

,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean,level,role
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890.000000,140900.000000,122395.000000,"[department, head, data, management, data, war...",department head data management data warehouse...,head,data engineer
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,64538.000000,82615.833333,73576.916667,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid,data analyst
3,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,join us let care for tomorrow at allianz globa...,en,11-04-2022,3,73200.000000,73931.500000,73565.750000,"[accumulation, data, engineer, accumulation, c...",accumulation data engineer accumulation catast...,mid,data engineer
4,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,we take responsibility for the sustainable man...,de,25-05-2022,4,70522.064556,68597.262764,69559.663660,"[actuarial, data, scientist]",actuarial data scientist,mid,data scientist
5,actuarial data scientist actuar,concordia versicherungsgesellschaft ag,hannover,versicherungen,concordia versicherungsgesellschaft ag,0.0,hirschau central full time about us conrad con...,de,11-04-2022,5,64172.000000,84575.000000,74373.500000,"[actuarial, data, scientist, actuar]",actuarial data scientist actuar,mid,data scientist


In [16]:
# This code saves the custom salary dataframe to an excel file
# The dataframe includes the text ID, low salary, high salary, and mean salary
# The file is saved on the user's Desktop in the tableau data folder


df_custom_salary = df_furher_analysis[['text_id','salary_low','salary_high','salary_mean']]
df_custom_salary.to_excel('C:/Users/Aleksej Aikov/Desktop/Enablement/Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/03_tableau_custom_salary.xlsx',index=False)

In [19]:
# This code tokenizes the job descriptions and removes stop words and punctuation

# The first line imports the stopwords from the nltk library. The stopwords are common words that don't convey meaning and can be removed from the text.
# The second line creates a list of stopwords and punctuation to be removed.
# The third line tokenizes the job descriptions using the word_tokenize function from the nltk library. This function breaks the text into individual words.
# The fourth line uses a list comprehension to remove the stopwords and punctuation from the tokenized text.
# The fifth line detokenizes the text using the TreebankWordDetokenizer function from the nltk library. This function puts the text back together into sentences.
# The sixth line returns the dataframe with the new columns.




def tokenazing_job_description(dataframe):

    stop_words = stopwords.words('english')
    to_remove2 = stop_words + list(string.punctuation)
    dataframe['job_desc_token'] = dataframe['jobDescription'].apply(word_tokenize).apply(lambda x: [item for item in x if item not in to_remove2])
    dataframe['job_desc_clean'] = dataframe['job_desc_token'].apply(TreebankWordDetokenizer().detokenize)
    return dataframe
    
df_furher_analysis = tokenazing_job_description(df_furher_analysis)
df_furher_analysis.head(5)

,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean,level,role,job_desc_token,job_desc_clean
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890.000000,140900.000000,122395.000000,"[department, head, data, management, data, war...",department head data management data warehouse...,head,data engineer,"[location, flexible, job, purpose, role, focus...",location flexible job purpose role focused add...
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,64538.000000,82615.833333,73576.916667,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid,data analyst,"[data, engineer, join, us, let, care, tomorrow...",data engineer join us let care tomorrow allian...
3,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,join us let care for tomorrow at allianz globa...,en,11-04-2022,3,73200.000000,73931.500000,73565.750000,"[accumulation, data, engineer, accumulation, c...",accumulation data engineer accumulation catast...,mid,data engineer,"[join, us, let, care, tomorrow, allianz, globa...",join us let care tomorrow allianz global inves...
4,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,we take responsibility for the sustainable man...,de,25-05-2022,4,70522.064556,68597.262764,69559.663660,"[actuarial, data, scientist]",actuarial data scientist,mid,data scientist,"[take, responsibility, sustainable, management...",take responsibility sustainable management ene...
5,actuarial data scientist actuar,concordia versicherungsgesellschaft ag,hannover,versicherungen,concordia versicherungsgesellschaft ag,0.0,hirschau central full time about us conrad con...,de,11-04-2022,5,64172.000000,84575.000000,74373.500000,"[actuarial, data, scientist, actuar]",actuarial data scientist actuar,mid,data scientist,"[hirschau, central, full, time, us, conrad, co...",hirschau central full time us conrad conquers ...


In [20]:
# The function "is_degree_required" takes in one argument (a dataframe) and returns a dataframe.
# This function is used to determine, based on the job description, whether a job requires a degree or not.

# The first step is to replace some words in the job descriptions with other words. This is done using the "replace" method.
# The reason for this is that some of the words that indicate that a degree is required are not always written in the same way.
# For example, the word "degree" can also be written as "completed studies" or "completed university degree".

# The second step is to use the "str.contains" method to check whether any of the replaced words are present in the job description.
# This method returns a boolean value (True or False).
# The result is then converted to an integer (0 or 1) using the "astype" method.

# Finally, the function returns the dataframe with an additional column called "degree_required".



def is_degree_required(dataframe):
    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(high degree of)", 'high level of ', regex = True)
    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(nfortatik )", 'informatics ', regex = True)
    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(you master)", 'you are mastering ', regex = True) 
    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(business informatist)", 'business informatics', regex = True) 
    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(informatist)", 'informatics ', regex = True) 
    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(wirtschaftswirtschaft|wirtschaft )", 'business', regex = True)
    dataframe['degree_required'] = dataframe['job_desc_clean'].str.contains("degree|completed studies|completed university degree|university degree|academic degree|completed study|study in the field|studies in the field", case=False, regex=True).astype(int)
    return dataframe
df_furher_analysis = is_degree_required(df_furher_analysis)
print(F'Number of jobs with degree mentioned: {df_furher_analysis[df_furher_analysis.degree_required == 1].shape[0]}')
print(F'Number of jobs without degree or not mentioned: {df_furher_analysis[df_furher_analysis.degree_required == 0].shape[0]}')


Number of jobs with degree mentioned: 1268
Number of jobs without degree or not mentioned: 1043


In [21]:
# The code below is taking the dataframe "df_furher_analysis" and adding three new columns: "bachelor", "master", and "phd". 
# The new columns each have a binary value (0 or 1) depending on whether the corresponding word appears in the job description field of that row. 
# The function returns the updated dataframe.

# The three print statements at the end are just calculating and printing the number of rows in which each word appears.


def finding_degree_required(dataframe):

    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(sciencebusiness intelligence)", 'business informatics', regex = True)
    dataframe['job_desc_clean'] = dataframe['job_desc_clean'].replace("(wirtschaftswirtschaft|wirtschaft)", 'business', regex = True)
    dataframe['bachelor'] = dataframe['job_desc_clean'].str.contains("bachelor|bachelor's|bachelors|bachelor degree|bachelor's degree", case=False, regex=True).astype(int)
    dataframe['phd'] = dataframe['job_desc_clean'].str.contains("phd|ph.d|phd degree|ph.d degree", case=False, regex=True).astype(int)
    dataframe['master'] = dataframe['job_desc_clean'].str.contains("master|master's|masters|master degree|masters degree|master's degree|master of business|master of science|master of arts", case=False, regex=True).astype(int)
    return dataframe
    
df_furher_analysis = finding_degree_required(df_furher_analysis)

print(F'Number of jobs with bachelor degree mentioned: {df_furher_analysis[df_furher_analysis.bachelor == 1].shape[0]}')
print(F'Number of jobs with master degree mentioned: {df_furher_analysis[df_furher_analysis.master == 1].shape[0]}')
print(F'Number of jobs with phd degree mentioned: {df_furher_analysis[df_furher_analysis.phd == 1].shape[0]}')


Number of jobs with bachelor degree mentioned: 177
Number of jobs with master degree mentioned: 401
Number of jobs with phd degree mentioned: 82


In [22]:

# This code is creating new columns in the dataframe to indicate which degree is required for each job.

# The first column, 'degree_new', is set to 1 if any degree is required for the job, and 0 if no degree is required.

# The second column, 'bachelor_new', is set to 1 if a bachelor's degree is required for the job, and 0 if not.

# The third column, 'master_new', is set to 1 if a master's degree is required for the job, and 0 if not.

# The fourth column, 'phd_new', is set to 1 if a PhD is required for the job, and 0 if not.

# The original columns ('degree_required', 'bachelor', 'master', and 'phd') are then dropped, and the new columns are renamed to match the original column names.


def cleaning_degree_and_degree_required(dataframe):

    dataframe['degree_new']=dataframe.apply(lambda x : 1 if x['degree_required']!=0 or x['bachelor']!=0 or x['master']!=0 or x['phd']!=0 else 0, axis=1)
    dataframe['bachelor_new']=dataframe.apply(lambda x : 1 if x['degree_new']==1 and x['master']==0 and x['phd']==0 else 0, axis=1)
    dataframe['master_new']=dataframe.apply(lambda x : 1 if x['degree_new']==1 and x['bachelor_new']==0 and x['phd']==0 else 0, axis=1)
    dataframe['phd_new']=dataframe.apply(lambda x : 1 if x['degree_new']==1 and x['bachelor_new']==0 and x['master_new']==0 else 0, axis=1)
    dataframe = dataframe.drop(columns=['degree_required', 'bachelor', 'master', 'phd'])
    dataframe.rename(columns={'degree_new': 'degree_required', 'bachelor_new': 'bachelor', 'master_new': 'master', 'phd_new': 'phd'}, inplace=True)

    return dataframe

df_furher_analysis = cleaning_degree_and_degree_required(df_furher_analysis)




print(F'Number of jobs with degree mentioned: {df_furher_analysis[df_furher_analysis.degree_required == 1].shape[0]}')
print(F'Number of jobs with bachelor degree mentioned: {df_furher_analysis[df_furher_analysis.bachelor == 1].shape[0]}')
print(F'Number of jobs with master degree mentioned: {df_furher_analysis[df_furher_analysis.master == 1].shape[0]}')
print(F'Number of jobs with phd degree mentioned: {df_furher_analysis[df_furher_analysis.phd == 1].shape[0]}')

df_furher_analysis.head(5)

Number of jobs with degree mentioned: 1416
Number of jobs with bachelor degree mentioned: 985
Number of jobs with master degree mentioned: 349
Number of jobs with phd degree mentioned: 82


,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,...,job_title_token,job_title_clean,level,role,job_desc_token,job_desc_clean,degree_required,bachelor,master,phd
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,...,"[department, head, data, management, data, war...",department head data management data warehouse...,head,data engineer,"[location, flexible, job, purpose, role, focus...",location flexible job purpose role focused add...,1,0,0,1
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,...,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid,data analyst,"[data, engineer, join, us, let, care, tomorrow...",data engineer join us let care tomorrow allian...,1,0,1,0
3,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,join us let care for tomorrow at allianz globa...,en,11-04-2022,3,...,"[accumulation, data, engineer, accumulation, c...",accumulation data engineer accumulation catast...,mid,data engineer,"[join, us, let, care, tomorrow, allianz, globa...",join us let care tomorrow allianz global inves...,1,0,1,0
4,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,we take responsibility for the sustainable man...,de,25-05-2022,4,...,"[actuarial, data, scientist]",actuarial data scientist,mid,data scientist,"[take, responsibility, sustainable, management...",take responsibility sustainable management ene...,1,1,0,0
5,actuarial data scientist actuar,concordia versicherungsgesellschaft ag,hannover,versicherungen,concordia versicherungsgesellschaft ag,0.0,hirschau central full time about us conrad con...,de,11-04-2022,5,...,"[actuarial, data, scientist, actuar]",actuarial data scientist actuar,mid,data scientist,"[hirschau, central, full, time, us, conrad, co...",hirschau central full time us conrad conquers ...,1,0,1,0


In [23]:
# This code is creating new columns in the dataframe df_further_analysis. 
# The new columns are related to different study fields, such as mathematics, statistics, physics, chemistry, biology, economics, business, computer science, economic engineering, business informatics, information management, information technology, computer engineering and computer science engineering. 
# For each of these columns, the code is checking if the job_desc_clean (which is a column in the dataframe) contains certain keywords related to the study field. 
# If the job_desc_clean contains the keyword, then the code will add a 1 to the column. If the job_desc_clean does not contain the keyword, then the code will add a 0 to the column.




def major_study_fields(dataframe):
    
    dataframe['mathematics'] = dataframe['job_desc_clean'].str.contains('math|mathematics', case=False, regex=True).astype(int)
    dataframe['statistics'] = dataframe['job_desc_clean'].str.contains('statistics', case=False, regex=True).astype(int)
    dataframe['physics'] = dataframe['job_desc_clean'].str.contains('physics', case=False, regex=True).astype(int)
    dataframe['chemistry'] = dataframe['job_desc_clean'].str.contains('chemistry', case=False, regex=True).astype(int)
    dataframe['biology'] = dataframe['job_desc_clean'].str.contains('biology', case=False, regex=True).astype(int)
    dataframe['economics'] = dataframe['job_desc_clean'].str.contains('economics|economic science', case=False, regex=True).astype(int)
    dataframe['business'] = dataframe['job_desc_clean'].str.contains('business administration|business economics|business sciences|bwl', case=False, regex=True).astype(int)
    dataframe['computer_science'] = dataframe['job_desc_clean'].str.contains('computer science|informatics|software engineering|informatics', case=False, regex=True).astype(int)
    dataframe['economic_engineering'] = dataframe['job_desc_clean'].str.contains('economic engineering|economic engineering system', case=False, regex=True).astype(int)
    dataframe['business_informatics'] = dataframe['job_desc_clean'].str.contains('business informatics', case=False, regex=True).astype(int)
    dataframe['information_management'] = dataframe['job_desc_clean'].str.contains('information management', case=False, regex=True).astype(int)
    dataframe['information_technology'] = dataframe['job_desc_clean'].str.contains('information technology', case=False, regex=True).astype(int)
    dataframe['computer_engineering'] = dataframe['job_desc_clean'].str.contains('computer engineering', case=False, regex=True).astype(int)
    dataframe['computer_science engineering'] = dataframe['job_desc_clean'].str.contains('computer science engineering', case=False, regex=True).astype(int)
    
    return dataframe

df_furher_analysis = major_study_fields(df_furher_analysis)


In [24]:
print(F'Number of jobs with mathematics mentioned: {df_furher_analysis[df_furher_analysis.mathematics == 1].shape[0]}')
print(F'Number of jobs with statistics mentioned: {df_furher_analysis[df_furher_analysis.statistics == 1].shape[0]}')
print(F'Number of jobs with physics mentioned: {df_furher_analysis[df_furher_analysis.physics == 1].shape[0]}')
print(F'Number of jobs with chemistry mentioned: {df_furher_analysis[df_furher_analysis.chemistry == 1].shape[0]}')
print(F'Number of jobs with biology mentioned: {df_furher_analysis[df_furher_analysis.biology == 1].shape[0]}')
print(F'Number of jobs with economics mentioned: {df_furher_analysis[df_furher_analysis.economics == 1].shape[0]}')
print(F'Number of jobs with business mentioned: {df_furher_analysis[df_furher_analysis.business == 1].shape[0]}')
print(F'Number of jobs with computer science mentioned: {df_furher_analysis[df_furher_analysis.computer_science == 1].shape[0]}')
print(F'Number of jobs with economic engineering mentioned: {df_furher_analysis[df_furher_analysis.economic_engineering == 1].shape[0]}')
print(F'Number of jobs with business informatics mentioned: {df_furher_analysis[df_furher_analysis.business_informatics == 1].shape[0]}')
print(F'Number of jobs with information management mentioned: {df_furher_analysis[df_furher_analysis.information_management == 1].shape[0]}')
print(F'Number of jobs with information technology mentioned: {df_furher_analysis[df_furher_analysis.information_technology == 1].shape[0]}')
print(F'Number of jobs with computer engineering mentioned: {df_furher_analysis[df_furher_analysis.computer_engineering == 1].shape[0]}')

df.head(5)

Number of jobs with mathematics mentioned: 717
Number of jobs with statistics mentioned: 479
Number of jobs with physics mentioned: 226
Number of jobs with chemistry mentioned: 19
Number of jobs with biology mentioned: 10
Number of jobs with economics mentioned: 251
Number of jobs with business mentioned: 188
Number of jobs with computer science mentioned: 1257
Number of jobs with economic engineering mentioned: 16
Number of jobs with business informatics mentioned: 364
Number of jobs with information management mentioned: 17
Number of jobs with information technology mentioned: 85
Number of jobs with computer engineering mentioned: 5


,jobTitle,Company,Location,Industry,CompanyGroup,Rating,jobDescription,org_lang,postDate,text_id,salary_low,salary_high,salary_mean,job_title_token,job_title_clean,level,role
1,department head data management data warehouse...,dfv deutsche familienversicherung ag,frankfurt,versicherungen,dfv,0.0,location is flexible job purpose the role is f...,en,25-05-2022,1,103890,140900,122395.0,"[department, head, data, management, data, war...",department head data management data warehouse...,head,data engineer
2,accumulation data analyst in accumulation and ...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,data engineer join us let care for tomorrow at...,en,11-04-2022,2,-1,-1,-1.0,"[accumulation, data, analyst, accumulation, ca...",accumulation data analyst accumulation catastr...,mid,data analyst
3,accumulation data engineer in accumulation and...,allianz global corporate & specialty,münchen,versicherungen,allianz,0.0,join us let care for tomorrow at allianz globa...,en,11-04-2022,3,-1,-1,-1.0,"[accumulation, data, engineer, accumulation, c...",accumulation data engineer accumulation catast...,mid,data engineer
4,actuarial data scientist,verti versicherung ag,teltow,finanzdienstleister,verti versicherung ag,3.5,we take responsibility for the sustainable man...,de,25-05-2022,4,-1,-1,-1.0,"[actuarial, data, scientist]",actuarial data scientist,mid,data scientist
5,actuarial data scientist actuar,concordia versicherungsgesellschaft ag,hannover,versicherungen,concordia versicherungsgesellschaft ag,0.0,hirschau central full time about us conrad con...,de,11-04-2022,5,-1,-1,-1.0,"[actuarial, data, scientist, actuar]",actuarial data scientist actuar,mid,data scientist


In [25]:
# This code is extracting data from a dataframe called df_furher_analysis. The data that is being extracted is based on the columns listed in the code. The data is then being exported to an excel file.


df_degree_required = df_furher_analysis[['text_id', 'degree_required', 'bachelor', 'master', 'phd','mathematics', 'statistics', 'physics', 'chemistry', 'biology', 'economics', 'business', 'computer_science', 
                        'economic_engineering', 'business_informatics', 'information_management', 'information_technology', 'computer_engineering', 'computer_science engineering']]
df_degree_required.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/04_tableau_analysis_degree_required.xlsx', index=False)

In [26]:
# This code is creating a new dataframe called df_visualization that only contains the text_id and job_desc_clean columns from the df_furher_analysis dataframe. 

# The code is then creating a list of visualization skills called visualization. 

# The code then defines a function called find_visualization_skills that takes two parameters, dataframe and *skills. The * in front of skills allows the function to take an arbitrary number of parameters. 

# The function finds all of the visualization skills in the job_desc_clean column and creates a new column for each of the skills in the list called visualization. The function then returns the dataframe. 

# The code then applies the find_visualization_skills function to the df_visualization dataframe using the list of visualization skills as the input. 

# The code then creates a new dataframe called df_visualization_skills that only contains the text_id, visualization, and the skills from the list. 

# The code then exports the df_visualization_skills dataframe to an Excel file.


df_visualization = df_furher_analysis[['text_id','job_desc_clean']].copy()

visualization = ['chartio','looker', 'zoho analytics','fusioncharts','cluvio','datawrapper','plotly','matplotlib','seaborn','dash','bokeh','highcharts',
                'spotfire', 'rshiny', 'domo', 'webfocus','superset','google charts','sisense','chartblocks']
def find_visualization_skills(dataframe, *skills):

    dataframe['visualization'] = dataframe['job_desc_clean'].str.contains('visualisation|data visualisation|visualization|dashboarding|dashboard', case=False, regex=True).astype(int)
    for skill in skills:
        dataframe[skill] = dataframe['job_desc_clean'].str.contains(f'{skill}', case=False, regex=True).astype(int)
        dataframe['power_bi'] = dataframe['job_desc_clean'].str.contains('powerbi|power bi', case=False, regex=True).astype(int)
        dataframe['tableau'] = dataframe['job_desc_clean'].str.contains('tableau|tableau online', case=False, regex=True).astype(int)
        dataframe['google_data_studio'] = dataframe['job_desc_clean'].str.contains('google data studio|data studio', case=False, regex=True).astype(int)
        dataframe['qlik'] = dataframe['job_desc_clean'].str.contains('qlik|qlikview|qliksense|qlik sense', case=False, regex=True).astype(int)
        dataframe.rename(columns={'google charts':'google_charts', 'zoho analytics':'zoho_analytics'}, inplace=True)
    return dataframe

df_visualization = find_visualization_skills(df_visualization, *visualization)   

df_visualization_skills = df_visualization[['text_id', 'visualization','chartio','looker', 'zoho_analytics','fusioncharts','cluvio','datawrapper','plotly',
                                            'matplotlib','seaborn','dash','bokeh','highcharts','spotfire','rshiny','domo','webfocus','superset','google_data_studio','sisense',
                                            'chartblocks','power_bi','tableau','google_charts','qlik','chartblocks']].copy()
df_visualization_skills.head(5)

df_visualization_skills.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/05_tableau_analysis_visualization_skills.xlsx',index=False)


In [27]:
print(F'Number of jobs with visualization mentioned: {df_visualization[df_visualization.visualization == 1].shape[0]}')
print(F'Number of jobs with chartio mentioned: {df_visualization[df_visualization.chartio == 1].shape[0]}')
print(F'Number of jobs with looker mentioned: {df_visualization[df_visualization.looker == 1].shape[0]}')
print(F'Number of jobs with zoho analytics mentioned: {df_visualization[df_visualization.zoho_analytics == 1].shape[0]}')
print(F'Number of jobs with fusioncharts mentioned: {df_visualization[df_visualization.fusioncharts == 1].shape[0]}')
print(F'Number of jobs with cluvio mentioned: {df_visualization[df_visualization.cluvio == 1].shape[0]}')
print(F'Number of jobs with datawrapper mentioned: {df_visualization[df_visualization.datawrapper == 1].shape[0]}')
print(F'Number of jobs with plotly mentioned: {df_visualization[df_visualization.plotly == 1].shape[0]}')
print(F'Number of jobs with matplotlib mentioned: {df_visualization[df_visualization.matplotlib == 1].shape[0]}')
print(F'Number of jobs with seaborn mentioned: {df_visualization[df_visualization.seaborn == 1].shape[0]}')
print(F'Number of jobs with dash mentioned: {df_visualization[df_visualization.dash == 1].shape[0]}')
print(F'Number of jobs with bokeh mentioned: {df_visualization[df_visualization.bokeh == 1].shape[0]}')
print(F'Number of jobs with highcharts mentioned: {df_visualization[df_visualization.highcharts == 1].shape[0]}')
print(F'Number of jobs with spotfire mentioned: {df_visualization[df_visualization.spotfire == 1].shape[0]}')
print(F'Number of jobs with rshiny mentioned: {df_visualization[df_visualization.rshiny == 1].shape[0]}')
print(F'Number of jobs with domo mentioned: {df_visualization[df_visualization.domo == 1].shape[0]}')
print(F'Number of jobs with webfocus mentioned: {df_visualization[df_visualization.webfocus == 1].shape[0]}')
print(F'Number of jobs with superset mentioned: {df_visualization[df_visualization.superset == 1].shape[0]}')
print(F'Number of jobs with google data studio mentioned: {df_visualization[df_visualization.google_data_studio == 1].shape[0]}')
print(F'Number of jobs with sisense mentioned: {df_visualization[df_visualization.sisense == 1].shape[0]}')
print(F'Number of jobs with chartblocks mentioned: {df_visualization[df_visualization.chartblocks == 1].shape[0]}')
print(F'Number of jobs with power bi mentioned: {df_visualization[df_visualization.power_bi == 1].shape[0]}')
print(F'Number of jobs with tableau mentioned: {df_visualization[df_visualization.tableau == 1].shape[0]}')
print(F'Number of jobs with qlik mentioned: {df_visualization[df_visualization.qlik == 1].shape[0]}')
print(F'Number of jobs with google charts mentioned: {df_visualization[df_visualization.google_charts == 1].shape[0]}')
print(F'Number of jobs with zoho analytics mentioned: {df_visualization[df_visualization.zoho_analytics == 1].shape[0]}')



Number of jobs with visualization mentioned: 627
Number of jobs with chartio mentioned: 0
Number of jobs with looker mentioned: 36
Number of jobs with zoho analytics mentioned: 0
Number of jobs with fusioncharts mentioned: 0
Number of jobs with cluvio mentioned: 0
Number of jobs with datawrapper mentioned: 2
Number of jobs with plotly mentioned: 15
Number of jobs with matplotlib mentioned: 13
Number of jobs with seaborn mentioned: 5
Number of jobs with dash mentioned: 358
Number of jobs with bokeh mentioned: 0
Number of jobs with highcharts mentioned: 0
Number of jobs with spotfire mentioned: 5
Number of jobs with rshiny mentioned: 1
Number of jobs with domo mentioned: 0
Number of jobs with webfocus mentioned: 3
Number of jobs with superset mentioned: 6
Number of jobs with google data studio mentioned: 37
Number of jobs with sisense mentioned: 1
Number of jobs with chartblocks mentioned: 0
Number of jobs with power bi mentioned: 386
Number of jobs with tableau mentioned: 287
Number of 

In [28]:
# In this code, the df_furher_analysis dataframe is filtered to only include the text_id and job_desc_clean columns, and this new dataframe is saved as df_programming_languages. 

# A list of programming languages is created, and a function called find_programming_skills is defined. This function takes in a dataframe and one or more skills as parameters. For each skill in the list of skills, the function looks through the job_desc_clean column of the dataframe for that skill, and if it is found, a new column is created in the dataframe with the name of the skill and a value of 1. If the skill is not found, the value in the new column is 0. 

# The find_programming_skills function is then called on the df_programming_languages dataframe, using the list of programming languages as the skills parameter. 

# The df_programming_languages dataframe is then filtered again to only include the text_id and columns for each of the programming languages. This new dataframe is then saved as an excel file.


df_programming_languages = df_furher_analysis[['text_id','job_desc_clean']].copy()

programming_languages = ['python','java', 'javascript', 'sql', 'html','matlab', 'sas', 'spss', 'stata','scala','spark','sparksql','pyspark'] # r is not in the list because of regex issue

def find_programming_skills(dataframe, *skills):
    for skill in skills:
        dataframe[skill] = dataframe['job_desc_clean'].str.contains(f'{skill}', case=False, regex=True).astype(int)
    return dataframe


df_programming_languages = find_programming_skills(df_programming_languages, *programming_languages)

df_programming_languages = df_programming_languages[['text_id','python','java', 'javascript', 'sql', 'html','matlab', 'sas', 'spss', 'stata','scala','spark','sparksql','pyspark']].copy()
df_programming_languages.head(5)

df_programming_languages.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/06_tableau_analysis_programming_languages.xlsx',index=False)

In [29]:
print(F'Number of jobs with python mentioned: {df_programming_languages[df_programming_languages.python == 1].shape[0]}')
print(F'Number of jobs with java mentioned: {df_programming_languages[df_programming_languages.java == 1].shape[0]}')
print(F'Number of jobs with javascript mentioned: {df_programming_languages[df_programming_languages.javascript == 1].shape[0]}')
print(F'Number of jobs with sql mentioned: {df_programming_languages[df_programming_languages.sql == 1].shape[0]}')
print(F'Number of jobs with html mentioned: {df_programming_languages[df_programming_languages.html == 1].shape[0]}')
print(F'Number of jobs with matlab mentioned: {df_programming_languages[df_programming_languages.matlab == 1].shape[0]}')
print(F'Number of jobs with sas mentioned: {df_programming_languages[df_programming_languages.sas == 1].shape[0]}')
print(F'Number of jobs with spss mentioned: {df_programming_languages[df_programming_languages.spss == 1].shape[0]}')
print(F'Number of jobs with stata mentioned: {df_programming_languages[df_programming_languages.stata == 1].shape[0]}')
print(F'Number of jobs with scala mentioned: {df_programming_languages[df_programming_languages.scala == 1].shape[0]}')
print(F'Number of jobs with spark mentioned: {df_programming_languages[df_programming_languages.spark == 1].shape[0]}')
print(F'Number of jobs with sparksql mentioned: {df_programming_languages[df_programming_languages.sparksql == 1].shape[0]}')
print(F'Number of jobs with pyspark mentioned: {df_programming_languages[df_programming_languages.pyspark == 1].shape[0]}')

Number of jobs with python mentioned: 1102
Number of jobs with java mentioned: 353
Number of jobs with javascript mentioned: 92
Number of jobs with sql mentioned: 1145
Number of jobs with html mentioned: 36
Number of jobs with matlab mentioned: 47
Number of jobs with sas mentioned: 119
Number of jobs with spss mentioned: 27
Number of jobs with stata mentioned: 17
Number of jobs with scala mentioned: 326
Number of jobs with spark mentioned: 298
Number of jobs with sparksql mentioned: 0
Number of jobs with pyspark mentioned: 24


In [30]:
# This code creates a new dataframe, df_platforms, which contains the text_id and job_desc_clean columns from the df_furher_analysis dataframe.

# Then, it defines a list of platforms, which are AWS, Azure, Snowflake, and Databricks.

# Next, it defines a function, platforms, which takes a dataframe and a list of skills as arguments. This function looks for each of the skills in the job description field of the dataframe, and if it finds a match, it creates a new column in the dataframe with a 1 in the row where the match occurred, and a 0 otherwise. The function also looks for matches for the Google BigQuery platform, and does the same thing.

# Finally, the code applies the platforms function to the df_platforms dataframe, using the list of platforms as the list of skills to look for.

# The code then creates a new dataframe, df_platforms, which contains the text_id, AWS, Azure, Snowflake, Databricks, and Google columns.

# The code then saves the df_platforms dataframe to an Excel file.



df_platforms = df_furher_analysis[['text_id','job_desc_clean']].copy()

list_of_platforms = ['aws','azure','snowflake','databricks']


def platforms(dataframe, *skills):
    for skill in skills:
        dataframe[skill] = dataframe['job_desc_clean'].str.contains(f'{skill}', case=False, regex=True).astype(int)
        dataframe['google'] = dataframe['job_desc_clean'].str.contains('google bigquery|bigquery|google cloud', case=False, regex=True).astype(int)
    return dataframe
df_platforms = platforms(df_platforms, *list_of_platforms)


df_platforms = df_platforms[['text_id','aws','azure','snowflake','databricks','google']].copy()
df_platforms.head(5)
df_platforms.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/07_tableau_analysis_platforms.xlsx',index=False)

In [31]:
print(F'Number of jobs with aws mentioned: {df_platforms[df_platforms.aws == 1].shape[0]}')
print(F'Number of jobs with azure mentioned: {df_platforms[df_platforms.azure == 1].shape[0]}')
print(F'Number of jobs with snowflake mentioned: {df_platforms[df_platforms.snowflake == 1].shape[0]}')
print(F'Number of jobs with databricks mentioned: {df_platforms[df_platforms.databricks == 1].shape[0]}')
print(F'Number of jobs with google mentioned: {df_platforms[df_platforms.google == 1].shape[0]}')


Number of jobs with aws mentioned: 367
Number of jobs with azure mentioned: 341
Number of jobs with snowflake mentioned: 71
Number of jobs with databricks mentioned: 84
Number of jobs with google mentioned: 139


In [32]:
# 1. The code creates a new dataframe called df_technologies which contains the columns 'text_id' and 'job_desc_clean' from the df_furher_analysis dataframe.
# 2. The code defines a list of technologies.
# 3. The code defines a function called find_technologies which takes two arguments, a dataframe and a list of technologies.
# 4. The function loops through the list of technologies and for each technology:
# 4.1. Creates a new column in the dataframe with the name of the technology.
# 4.2. Fills the new column with '1' if the technology is mentioned in the job description and '0' if it is not.
# 5. The code applies the function to the df_technologies dataframe using the list of technologies defined in step 2.
# 6. The code creates a new dataframe called df_technologies which contains the columns from step 5.
# 7. The code exports the df_technologies dataframe to an Excel file.



df_technologies = df_furher_analysis[['text_id','job_desc_clean']].copy()

technologies = ['bigsql','cassandra','hadoop','hbase','hdfs','hive','hivesql','trifacta',
                'kubernetes','mongodb','mysql','nosql','tsql','s3','redshift','posgres',
                'teradata','terraform','alteryx','excel','powerapps','powerpoint','knime','talend',
                'y42','fivetran','matillion','pentaho','kafka', 'luigi', 'airflow','etl',
                'database','dbt','dynamo','firebase','firestore','git','github','gitlab','bigquery']


def find_technologies(dataframe, *skills):

    for skill in skills:
        dataframe[skill] = dataframe['job_desc_clean'].str.contains(f'{skill}', case=False, regex=True).astype(int)
        dataframe['aws_glue'] = dataframe['job_desc_clean'].str.contains('aws glue', case=False, regex=True).astype(int)
        dataframe['data_vault'] = dataframe['job_desc_clean'].str.contains('data vault', case=False, regex=True).astype(int)
        dataframe['docker'] = dataframe['job_desc_clean'].str.contains('docker|dockering', case=False, regex=True).astype(int)
        dataframe['data_lake'] = dataframe['job_desc_clean'].str.contains('data lake', case=False, regex=True).astype(int)
        dataframe['azure_synapse_analytics'] = dataframe['job_desc_clean'].str.contains('synapse analytics', case=False, regex=True).astype(int)
        dataframe['azure_data_factory'] = dataframe['job_desc_clean'].str.contains('data factory', case=False, regex=True).astype(int)
        dataframe['microsoft_office'] = dataframe['job_desc_clean'].str.contains('microsoft office', case=False, regex=True).astype(int)

    return dataframe

df_technologies = find_technologies(df_technologies, *technologies)

df_technologies = df_technologies[['text_id','bigsql','cassandra','hadoop','hbase','hdfs','hive','hivesql','trifacta','kubernetes',
                                    'mongodb','mysql','nosql','tsql','s3','redshift','posgres','teradata','terraform','alteryx','excel','powerapps',
                                    'powerpoint','knime','talend','y42','fivetran','matillion','pentaho','kafka', 'luigi', 'airflow','etl','database',
                                    'dbt','dynamo','firebase','firestore','git','github','gitlab','aws_glue','data_vault','docker',
                                    'azure_synapse_analytics','azure_data_factory','microsoft_office','data_lake','bigquery']].copy()

df_technologies.head(5)

df_technologies.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/08_tableau_analysis_technologies.xlsx',index=False)


In [33]:
print(F'Number of jobs with bigsql mentioned: {df_technologies[df_technologies.bigsql == 1].shape[0]}')
print(F'Number of jobs with cassandra mentioned: {df_technologies[df_technologies.cassandra == 1].shape[0]}')
print(F'Number of jobs with hadoop mentioned: {df_technologies[df_technologies.hadoop == 1].shape[0]}')
print(F'Number of jobs with hbase mentioned: {df_technologies[df_technologies.hbase == 1].shape[0]}')
print(F'Number of jobs with hdfs mentioned: {df_technologies[df_technologies.hdfs == 1].shape[0]}')
print(F'Number of jobs with hive mentioned: {df_technologies[df_technologies.hive == 1].shape[0]}')
print(F'Number of jobs with hivesql mentioned: {df_technologies[df_technologies.hivesql == 1].shape[0]}')
print(F'Number of jobs with data lake mentioned: {df_technologies[df_technologies.data_lake == 1].shape[0]}')
print(F'Number of jobs with trifacta mentioned: {df_technologies[df_technologies.trifacta == 1].shape[0]}')
print(F'Number of jobs with kubernetes mentioned: {df_technologies[df_technologies.kubernetes == 1].shape[0]}')
print(F'Number of jobs with mongodb mentioned: {df_technologies[df_technologies.mongodb == 1].shape[0]}')
print(F'Number of jobs with mysql mentioned: {df_technologies[df_technologies.mysql == 1].shape[0]}')
print(F'Number of jobs with nosql mentioned: {df_technologies[df_technologies.nosql == 1].shape[0]}')
print(F'Number of jobs with tsql mentioned: {df_technologies[df_technologies.tsql == 1].shape[0]}')
print(F'Number of jobs with s3 mentioned: {df_technologies[df_technologies.s3 == 1].shape[0]}')
print(F'Number of jobs with data factory mentioned: {df_technologies[df_technologies.azure_data_factory == 1].shape[0]}')
print(F'Number of jobs with redshift mentioned: {df_technologies[df_technologies.redshift == 1].shape[0]}')
print(F'Number of jobs with posgres mentioned: {df_technologies[df_technologies.posgres == 1].shape[0]}')
print(F'Number of jobs with teradata mentioned: {df_technologies[df_technologies.teradata == 1].shape[0]}')
print(F'Number of jobs with terraform mentioned: {df_technologies[df_technologies.terraform == 1].shape[0]}')
print(F'Number of jobs with alteryx mentioned: {df_technologies[df_technologies.alteryx == 1].shape[0]}')
print(F'Number of jobs with excel mentioned: {df_technologies[df_technologies.excel == 1].shape[0]}')
print(F'Number of jobs with powerapps mentioned: {df_technologies[df_technologies.powerapps == 1].shape[0]}')
print(F'Number of jobs with microsoft office mentioned: {df_technologies[df_technologies.microsoft_office == 1].shape[0]}')
print(F'Number of jobs with synapse analytics mentioned: {df_technologies[df_technologies.azure_synapse_analytics == 1].shape[0]}')
print(F'Number of jobs with powerpoint mentioned: {df_technologies[df_technologies.powerpoint == 1].shape[0]}')
print(F'Number of jobs with knime mentioned: {df_technologies[df_technologies.knime == 1].shape[0]}')
print(F'Number of jobs with talend mentioned: {df_technologies[df_technologies.talend == 1].shape[0]}')
print(F'Number of jobs with y42 mentioned: {df_technologies[df_technologies.y42 == 1].shape[0]}')
print(F'Number of jobs with fivetran mentioned: {df_technologies[df_technologies.fivetran == 1].shape[0]}')
print(F'Number of jobs with matillion mentioned: {df_technologies[df_technologies.matillion == 1].shape[0]}')
print(F'Number of jobs with pentaho mentioned: {df_technologies[df_technologies.pentaho == 1].shape[0]}')
print(F'Number of jobs with kafka mentioned: {df_technologies[df_technologies.kafka == 1].shape[0]}')
print(F'Number of jobs with luigi mentioned: {df_technologies[df_technologies.luigi == 1].shape[0]}')
print(F'Number of jobs with aws glue mentioned: {df_technologies[df_technologies.aws_glue == 1].shape[0]}')
print(F'Number of jobs with data vault mentioned: {df_technologies[df_technologies.data_vault == 1].shape[0]}')
print(F'Number of jobs with docker mentioned: {df_technologies[df_technologies.docker == 1].shape[0]}')
print(F'Number of jobs with bigquery mentioned: {df_technologies[df_technologies.bigquery == 1].shape[0]}')

Number of jobs with bigsql mentioned: 0
Number of jobs with cassandra mentioned: 14
Number of jobs with hadoop mentioned: 134
Number of jobs with hbase mentioned: 10
Number of jobs with hdfs mentioned: 7
Number of jobs with hive mentioned: 40
Number of jobs with hivesql mentioned: 0
Number of jobs with data lake mentioned: 121
Number of jobs with trifacta mentioned: 3
Number of jobs with kubernetes mentioned: 103
Number of jobs with mongodb mentioned: 39
Number of jobs with mysql mentioned: 69
Number of jobs with nosql mentioned: 107
Number of jobs with tsql mentioned: 4
Number of jobs with s3 mentioned: 0
Number of jobs with data factory mentioned: 35
Number of jobs with redshift mentioned: 55
Number of jobs with posgres mentioned: 0
Number of jobs with teradata mentioned: 8
Number of jobs with terraform mentioned: 60
Number of jobs with alteryx mentioned: 28
Number of jobs with excel mentioned: 638
Number of jobs with powerapps mentioned: 5
Number of jobs with microsoft office mentio

In [34]:

# 1) The first line copies the 'text_id' and 'job_desc_clean' columns from the df_furher_analysis dataframe into a new dataframe called df_softskill.

# 2) The second line creates a list of soft skills.

# 3) The third line defines a function called find_softskill. This function takes two arguments - a dataframe and a list of skills. For each skill in the list, the function adds a new column to the dataframe with the name of the skill. The values in the column are either 1 (indicating that the skill is mentioned in the job description) or 0 (indicating that the skill is not mentioned in the job description).

# 4) The fourth line applies the find_softskill function to the df_softskill dataframe, using the list of soft skills created in the second line as the list of skills.

# 5) The fifth line keeps only the columns that were created in the fourth line (i.e. the columns containing the soft skills).

# 6) The sixth line keeps only the columns that were created in the fourth line (i.e. the columns containing the soft skills) and saves them to a new Excel file.


df_softskill = df_furher_analysis[['text_id','job_desc_clean']].copy()

list_of_softskills = ['communication', 'verbal', 'written', 'oral','crossfunctional','crossorganizational', 'multifunctional', 'teamwork', 'collaboration','critical thinking','attention to detail','interpersonal']

def find_softskill(dataframe, *skills):
    for skill in skills:
        dataframe[skill] = dataframe['job_desc_clean'].str.contains(f'{skill}', case=False, regex=True).astype(int)
    dataframe['presentation_skills'] = dataframe['job_desc_clean'].str.contains('presentation skills', case=False, regex=True).astype(int)
    dataframe['critical_thinking'] = dataframe['job_desc_clean'].str.contains('critical thinking', case=False, regex=True).astype(int)
    dataframe['problem_solving'] = dataframe['job_desc_clean'].str.contains('problem solving', case=False, regex=True).astype(int)
    dataframe['decision_making'] = dataframe['job_desc_clean'].str.contains('decision making', case=False, regex=True).astype(int)
    dataframe['attention_to_detail'] = dataframe['job_desc_clean'].str.contains('attention to detail', case=False, regex=True).astype(int)
    dataframe['analytical_skills'] = dataframe['job_desc_clean'].str.contains('analytical skills', case=False, regex=True).astype(int)
    return dataframe
df_softskill = find_softskill(df_softskill, *list_of_softskills)

df_softskill = df_softskill[['text_id','communication', 'verbal', 'written', 'oral','crossfunctional','crossorganizational', 
                            'multifunctional', 'teamwork', 'collaboration','critical thinking','attention to detail','interpersonal',
                            'presentation_skills','critical_thinking','problem_solving','decision_making','attention_to_detail','analytical_skills']]


df_softskill = df_softskill[['text_id','communication', 'verbal', 'written', 'oral','crossfunctional','crossorganizational','multifunctional', 
                                'teamwork', 'collaboration','critical thinking','attention to detail','interpersonal','presentation_skills',
                                'critical_thinking','problem_solving','decision_making','attention_to_detail','analytical_skills']]

df_softskill.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/09_tableau_analysis_softskills.xlsx',index=False)

In [35]:
print(F'Number of jobs with communication mentioned: {df_softskill[df_softskill.communication == 1].shape[0]}')
print(F'Number of jobs with verbal mentioned: {df_softskill[df_softskill.verbal == 1].shape[0]}')
print(F'Number of jobs with written mentioned: {df_softskill[df_softskill.written == 1].shape[0]}')
print(F'Number of jobs with oral mentioned: {df_softskill[df_softskill.oral == 1].shape[0]}')
print(F'Number of jobs with crossfunctional mentioned: {df_softskill[df_softskill.crossfunctional == 1].shape[0]}')
print(F'Number of jobs with crossorganizational mentioned: {df_softskill[df_softskill.crossorganizational == 1].shape[0]}')
print(F'Number of jobs with multifunctional mentioned: {df_softskill[df_softskill.multifunctional == 1].shape[0]}')
print(F'Number of jobs with teamwork mentioned: {df_softskill[df_softskill.teamwork == 1].shape[0]}')
print(F'Number of jobs with collaboration mentioned: {df_softskill[df_softskill.collaboration == 1].shape[0]}')
print(F'Number of jobs with critical thinking mentioned: {df_softskill[df_softskill.critical_thinking == 1].shape[0]}')
print(F'Number of jobs with interpersonal mentioned: {df_softskill[df_softskill.interpersonal == 1].shape[0]}')
print(F'Number of jobs with attention to detail mentioned: {df_softskill[df_softskill.attention_to_detail == 1].shape[0]}')
print(F'Number of jobs with analytical skills mentioned: {df_softskill[df_softskill.analytical_skills == 1].shape[0]}')
print(F'Number of jobs with presentation skills mentioned: {df_softskill[df_softskill.presentation_skills == 1].shape[0]}')
print(F'Number of jobs with problem solving mentioned: {df_softskill[df_softskill.problem_solving == 1].shape[0]}')
print(F'Number of jobs with decision making mentioned: {df_softskill[df_softskill.decision_making == 1].shape[0]}')


Number of jobs with communication mentioned: 983
Number of jobs with verbal mentioned: 75
Number of jobs with written mentioned: 308
Number of jobs with oral mentioned: 83
Number of jobs with crossfunctional mentioned: 0
Number of jobs with crossorganizational mentioned: 0
Number of jobs with multifunctional mentioned: 2
Number of jobs with teamwork mentioned: 239
Number of jobs with collaboration mentioned: 179
Number of jobs with critical thinking mentioned: 5
Number of jobs with interpersonal mentioned: 35
Number of jobs with attention to detail mentioned: 0
Number of jobs with analytical skills mentioned: 170
Number of jobs with presentation skills mentioned: 69
Number of jobs with problem solving mentioned: 139
Number of jobs with decision making mentioned: 211


In [36]:
# This code takes a dataframe of job descriptions, creates bigrams for each job description, and then looks for certain bigrams that are related to machine learning skills.
#  For each machine learning skill, a new column is created in the dataframe, and the value in that column corresponds to whether or not that skill was found in the job description. Finally, the dataframe is exported to an excel file.


df_ml_skills = df_furher_analysis[['text_id','job_desc_clean']].copy()

def create_bigrams(dataframe):
    dataframe['job_desc_bigrams'] = dataframe['job_desc_clean'].apply(lambda x: list(nltk.bigrams(x.split())))
    return dataframe
df_ml_skills = create_bigrams(df_ml_skills)

ml_skills =  [('machine','learning'), ('predictive', 'modeling'), ('linear', 'regression'), ('logistic', 'regression'), ('k','means'), ('random', 'forest'), ('naive', 'bayes'), ('pca', 'svd'), ('decision', 'tree'), ('ensemble', 'model')]

def create_columns_for_ml_skills(dataframe, column_name, list_of_bigrams):
    for bigram in list_of_bigrams:
        dataframe[bigram] = dataframe[column_name].apply(lambda x: 1 if bigram in x else 0)
    
    return dataframe

df_ml_skills = create_columns_for_ml_skills(df_ml_skills, 'job_desc_bigrams', ml_skills)


def rename_ml_skills_columns(dataframe):

    dataframe = dataframe.rename(columns = {dataframe.columns[-1] : 'ensemble_model', dataframe.columns[-2] : 'decision_tree', dataframe.columns[-3] : 'pca_svd', dataframe.columns[-4] : 'naive_bayes', 
                                                dataframe.columns[-5] : 'random_forest', dataframe.columns[-6] : 'k_means', dataframe.columns[-7] : 'logistic_regression', dataframe.columns[-8] : 'linear_regression', 
                                                dataframe.columns[-9] : 'predictive_modeling', dataframe.columns[-10] : 'machine_learning'})
    return dataframe
df_ml_skills = rename_ml_skills_columns(df_ml_skills)


df_ml_skills = df_ml_skills[['text_id','ensemble_model', 'decision_tree', 'pca_svd', 'naive_bayes', 'random_forest', 'k_means', 'logistic_regression', 
                            'linear_regression', 'predictive_modeling', 'machine_learning']]
df_ml_skills.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/10_tableau_analysis_ml_skills.xlsx',index=False)


In [37]:
print(F'Number of jobs with ensemble model mentioned: {df_ml_skills[df_ml_skills.ensemble_model == 1].shape[0]}')
print(F'Number of jobs with decision tree mentioned: {df_ml_skills[df_ml_skills.decision_tree == 1].shape[0]}')
print(F'Number of jobs with pca svd mentioned: {df_ml_skills[df_ml_skills.pca_svd == 1].shape[0]}')
print(F'Number of jobs with naive bayes mentioned: {df_ml_skills[df_ml_skills.naive_bayes == 1].shape[0]}')
print(F'Number of jobs with random forest mentioned: {df_ml_skills[df_ml_skills.random_forest == 1].shape[0]}')
print(F'Number of jobs with k means mentioned: {df_ml_skills[df_ml_skills.k_means == 1].shape[0]}')
print(F'Number of jobs with logistic regression mentioned: {df_ml_skills[df_ml_skills.logistic_regression == 1].shape[0]}')
print(F'Number of jobs with linear regression mentioned: {df_ml_skills[df_ml_skills.linear_regression == 1].shape[0]}')
print(F'Number of jobs with predictive modeling mentioned: {df_ml_skills[df_ml_skills.predictive_modeling == 1].shape[0]}')
print(F'Number of jobs with machine learning mentioned: {df_ml_skills[df_ml_skills.machine_learning == 1].shape[0]}')


Number of jobs with ensemble model mentioned: 0
Number of jobs with decision tree mentioned: 8
Number of jobs with pca svd mentioned: 0
Number of jobs with naive bayes mentioned: 1
Number of jobs with random forest mentioned: 10
Number of jobs with k means mentioned: 0
Number of jobs with logistic regression mentioned: 4
Number of jobs with linear regression mentioned: 1
Number of jobs with predictive modeling mentioned: 24
Number of jobs with machine learning mentioned: 594


In [38]:

# This code is creating a dataframe with job descriptions and the corresponding text ID. It then uses the function "create_bigrams" to create a column of bigrams for each job description. 
# The function "create_columns_for_deep_learning_skills" then creates a column for each of the deep learning skills listed in "list_of_deep_learning_skills" and puts a 1 in the column if the skill is mentioned in the job description, and a 0 if it is not. 
# The function "rename_deep_learning_skills_columns" then renames the columns so that they are more easily interpretable. Finally, the code outputs the dataframe to an Excel file.


df_deep_learning_skills = df_furher_analysis[['text_id','job_desc_clean']].copy()

def create_bigrams(dataframe):
    dataframe['job_desc_bigrams'] = dataframe['job_desc_clean'].apply(lambda x: list(nltk.bigrams(x.split())))
    return dataframe
deep_learning_skills_df = create_bigrams(df_deep_learning_skills)

list_of_deep_learning_skills = [('neural','network'), ('deep', 'learning'), ('object', 'detection'), ('keras', 'tensorflow'), ('convolutional', 'neural')]

def create_columns_for_deep_learning_skills(dataframe, column_name, list_of_bigrams):
    for bigram in list_of_bigrams:
        dataframe[bigram] = dataframe[column_name].apply(lambda x: 1 if bigram in x else 0)
    return dataframe
deep_learning_skills_df = create_columns_for_deep_learning_skills(deep_learning_skills_df, 'job_desc_bigrams', list_of_deep_learning_skills)

def rename_deep_learning_skills_columns(dataframe):
    dataframe = dataframe.rename(columns = {dataframe.columns[-1] : 'object_detection', dataframe.columns[-2] : 'convolutional_neural', dataframe.columns[-3] : 'keras_tensorflow', dataframe.columns[-4] : 'neural_network', dataframe.columns[-5] : 'deep_learning'})
    return dataframe
df_deep_learning_skills = rename_deep_learning_skills_columns(deep_learning_skills_df)


df_deep_learning_skills = df_deep_learning_skills[['text_id','object_detection', 'convolutional_neural', 'keras_tensorflow', 'neural_network', 'deep_learning']]

df_deep_learning_skills.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/11_tableau_analysis_deep_learning_skills.xlsx',index=False)

In [39]:
print(F'Number of jobs with neural network mentioned: {df_deep_learning_skills[df_deep_learning_skills.neural_network == 1].shape[0]}')
print(F'Number of jobs with deep learning mentioned: {df_deep_learning_skills[df_deep_learning_skills.deep_learning == 1].shape[0]}')
print(F'Number of jobs with object detection mentioned: {df_deep_learning_skills[df_deep_learning_skills.object_detection == 1].shape[0]}')
print(F'Number of jobs with keras tensorflow mentioned: {df_deep_learning_skills[df_deep_learning_skills.keras_tensorflow == 1].shape[0]}')
print(F'Number of jobs with convolutional neural mentioned: {df_deep_learning_skills[df_deep_learning_skills.convolutional_neural == 1].shape[0]}')


Number of jobs with neural network mentioned: 91
Number of jobs with deep learning mentioned: 3
Number of jobs with object detection mentioned: 0
Number of jobs with keras tensorflow mentioned: 0
Number of jobs with convolutional neural mentioned: 12


In [40]:
# This code creates a new dataframe, df_languages_required, that contains the text_id and job_desc_clean columns from the original dataframe, df_furher_analysis.

# A list of languages, list_of_language_required, is created.

# A function, find_language, is created that takes a dataframe and a list of languages as arguments. The function searches the job_desc_clean column of the dataframe for each language in the list and creates a new column in the dataframe for each language. The function returns the dataframe.

# The function is called on the df_languages_required dataframe using the list of languages as arguments. This creates new columns in the df_languages_required dataframe for each language in the list.

# The df_languages_required dataframe is filtered to only contain the text_id and language columns.


df_languages_required = df_furher_analysis[['text_id','job_desc_clean']].copy()

list_of_language_required = ['english', 'french', 'spanish', 'german', 'italian', 'portuguese', 'russian', 'chinese', 'japanese', 'ukrainian', 'arabic', 'hebrew']

def find_language(dataframe, *languages):
    for language in languages:
        dataframe[language] = dataframe['job_desc_clean'].str.contains(f'{language}', case=False, regex=True).astype(int)
    return dataframe
df_languages_required = find_language(df_languages_required, *list_of_language_required)


df_languages_required = df_languages_required[['text_id','english', 'french', 'spanish', 'german', 'italian', 'portuguese', 'russian', 'chinese', 'japanese', 'ukrainian', 'arabic', 'hebrew']]

df_languages_required.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/12_tableau_analysis_languages_required.xlsx',index=False)


In [41]:
print(F'Number of jobs with english mentioned: {df_languages_required[df_languages_required.english == 1].shape[0]}')
print(F'Number of jobs with french mentioned: {df_languages_required[df_languages_required.french == 1].shape[0]}')
print(F'Number of jobs with spanish mentioned: {df_languages_required[df_languages_required.spanish == 1].shape[0]}')
print(F'Number of jobs with german mentioned: {df_languages_required[df_languages_required.german == 1].shape[0]}')
print(F'Number of jobs with italian mentioned: {df_languages_required[df_languages_required.italian == 1].shape[0]}')
print(F'Number of jobs with portuguese mentioned: {df_languages_required[df_languages_required.portuguese == 1].shape[0]}')
print(F'Number of jobs with russian mentioned: {df_languages_required[df_languages_required.russian == 1].shape[0]}')
print(F'Number of jobs with chinese mentioned: {df_languages_required[df_languages_required.chinese == 1].shape[0]}')
print(F'Number of jobs with japanese mentioned: {df_languages_required[df_languages_required.japanese == 1].shape[0]}')
print(F'Number of jobs with ukrainian mentioned: {df_languages_required[df_languages_required.ukrainian == 1].shape[0]}')
print(F'Number of jobs with arabic mentioned: {df_languages_required[df_languages_required.arabic == 1].shape[0]}')
print(F'Number of jobs with hebrew mentioned: {df_languages_required[df_languages_required.hebrew == 1].shape[0]}')


Number of jobs with english mentioned: 1329
Number of jobs with french mentioned: 19
Number of jobs with spanish mentioned: 5
Number of jobs with german mentioned: 1359
Number of jobs with italian mentioned: 2
Number of jobs with portuguese mentioned: 0
Number of jobs with russian mentioned: 2
Number of jobs with chinese mentioned: 1
Number of jobs with japanese mentioned: 2
Number of jobs with ukrainian mentioned: 1
Number of jobs with arabic mentioned: 0
Number of jobs with hebrew mentioned: 0


In [42]:
# This code is creating a new dataframe that only contains the "text_id" and "job_desc_clean" columns from the "df_furher_analysis" dataframe. 
# This new dataframe is called "df_projectsmanagement_skills_and_methodologies".

# There is also a list called "list_of_projectmanagement_skills_and_methodologies" which contains a list of project management skills and methodologies. 

# There is a function called "find_projectmanagement_skills_and_methodologies" which takes in a dataframe and a list of project management skills and methodologies. 
# This function looks through the "job_desc_clean" column in the dataframe and looks for each of the project management skills and methodologies in the list. 
# If it finds a match, it will create a new column in the dataframe with the name of the skill or methodology and put a "1" in the cells of that column for each row where there was a match.
# If it doesn't find a match, it will create a new column in the dataframe with the name of the skill or methodology and put a "0" in the cells of that column for each row where there wasn't a match.

# The function also looks for other project management skills and methodologies that weren't in the list (team management, organization management, project planning, time management, project scheduling, and project budgeting)
# and does the same thing. 

# After the function is done running, it returns the dataframe. 

# The code then takes the dataframe that was returned from the function and puts it into the "df_projectsmanagement_skills_and_methodologies" dataframe. 
# It also only keeps the columns that we care about and gets rid of the rest. 

# Finally, the code exports the dataframe to an Excel file.




df_projectsmanagement_skills_and_methodologies= df_furher_analysis[['text_id','job_desc_clean']].copy()


list_of_projectmanagement_skills_and_methodologies = ['negotiation','leadership','waterfall','agile', 'scrum','lean','adaptability','patience']

def find_projectmanagement_skills_and_methodologies(dataframe, *list_of_projectmanagement_skills_and_methodologies):
    for skill in list_of_projectmanagement_skills_and_methodologies:
        dataframe[skill] = dataframe['job_desc_clean'].str.contains(f'{skill}', case=False, regex=True).astype(int)

    dataframe['team_management'] = dataframe['job_desc_clean'].str.contains('team_management', case=False, regex=True).astype(int)
    dataframe['organization_management'] = dataframe['job_desc_clean'].str.contains('organization management', case=False, regex=True).astype(int)
    dataframe['project_planning'] = dataframe['job_desc_clean'].str.contains('project planning', case=False, regex=True).astype(int)
    dataframe['time_management'] = dataframe['job_desc_clean'].str.contains('time management', case=False, regex=True).astype(int)
    dataframe['project_scheduling'] = dataframe['job_desc_clean'].str.contains('project scheduling', case=False, regex=True).astype(int)
    dataframe['project_budgeting'] = dataframe['job_desc_clean'].str.contains('project budgeting', case=False, regex=True).astype(int)
    dataframe['prince2'] = dataframe['job_desc_clean'].str.contains('prince', case=False, regex=True).astype(int)
    return dataframe

df_projectsmanagement_skills_and_methodologies = find_projectmanagement_skills_and_methodologies(df_projectsmanagement_skills_and_methodologies, *list_of_projectmanagement_skills_and_methodologies)
df_projectsmanagement_skills_and_methodologies

df_projectsmanagement_skills_and_methodologies = df_projectsmanagement_skills_and_methodologies[['text_id','negotiation', 'leadership', 'waterfall', 'lean','agile', 'scrum', 'prince2', 
                                                'adaptability', 'patience', 'team_management', 'organization_management', 'project_planning', 'time_management', 
                                                'project_scheduling', 'project_budgeting']]


df_projectsmanagement_skills_and_methodologies.to_excel('Master/04_Analysis/04_01_custom_analysis/04_01_02_tableau_data/13_tableau_analysis_project_management_skills_and_methodologies.xlsx',index=False)

In [43]:
print(F'Number of jobs with negotiation mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.negotiation == 1].shape[0]}')
print(F'Number of jobs with leadership mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.leadership == 1].shape[0]}')
print(F'Number of jobs with waterfall mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.waterfall == 1].shape[0]}')
print(F'Number of jobs with agile mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.agile == 1].shape[0]}')
print(F'Number of jobs with scrum mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.scrum == 1].shape[0]}')
print(F'Number of jobs with lean mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.lean == 1].shape[0]}')
print(F'Number of jobs with prince2 mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.prince2 == 1].shape[0]}')
print(F'Number of jobs with adaptability mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.adaptability == 1].shape[0]}')
print(F'Number of jobs with patience mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.patience == 1].shape[0]}')
print(F'Number of jobs with time_management mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.time_management == 1].shape[0]}')
print(F'Number of jobs with organization_management mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.organization_management == 1].shape[0]}')
print(F'Number of jobs with project_planning mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.project_planning == 1].shape[0]}')
print(F'Number of jobs with project_scheduling mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.project_scheduling == 1].shape[0]}')
print(F'Number of jobs with project_budgeting mentioned: {df_projectsmanagement_skills_and_methodologies[df_projectsmanagement_skills_and_methodologies.project_budgeting == 1].shape[0]}')


Number of jobs with negotiation mentioned: 41
Number of jobs with leadership mentioned: 127
Number of jobs with waterfall mentioned: 11
Number of jobs with agile mentioned: 694
Number of jobs with scrum mentioned: 117
Number of jobs with lean mentioned: 148
Number of jobs with prince2 mentioned: 6
Number of jobs with adaptability mentioned: 3
Number of jobs with patience mentioned: 1
Number of jobs with time_management mentioned: 17
Number of jobs with organization_management mentioned: 1
Number of jobs with project_planning mentioned: 12
Number of jobs with project_scheduling mentioned: 0
Number of jobs with project_budgeting mentioned: 0
